In [3]:
import pylab
import skimage 

Загрузите картинку parrots.jpg. Преобразуйте изображение, приведя все значения в интервал от 0 до 1. Для этого можно воспользоваться функцией img_as_float из модуля skimage. Обратите внимание на этот шаг, так как при работе с исходным изображением вы получите некорректный результат.

In [8]:
img = skimage.img_as_float('parrots.jpg')

ValueError: Can not convert from |S11 to float64.